In [ ]:
# 2) Dataset scan + build data.yaml for pose (robust scan)
from pathlib import Path
import os, yaml
from glob import glob
from collections import Counter

root = Path('./dataset')
assert root.exists(), "Dataset folder './dataset' not found. Please scp/rsync it first."

# Find YOLO label .txt anywhere under labels/ (supports both layouts)
label_txts = []
label_txts += glob(str(root / 'labels' / '**' / '*.txt'), recursive=True)          # type-first: dataset/labels/train/*.txt
label_txts += glob(str(root / '*' / 'labels' / '**' / '*.txt'), recursive=True)    # split-first: dataset/train/labels/*.txt
label_txts = sorted(set(label_txts))
print(f"Found {len(label_txts)} label files under 'labels' folders")

if len(label_txts) == 0:
    # Last resort: search any .txt and warn
    label_txts = glob(str(root / '**' / '*.txt'), recursive=True)
    print(f"Fallback: found {len(label_txts)} .txt files total")
    assert len(label_txts)>0, "No label .txt files found anywhere in './dataset'. Expected YOLO-pose labels under .../labels/..."

# Robustly infer keypoints K by majority across many labels
kp_counter = Counter()
for i, fp in enumerate(label_txts[:2000]):  # sample up to 2000 files
    try:
        with open(fp, 'r') as f:
            for j, ln in enumerate(f):
                ln = ln.strip()
                if not ln:
                    continue
                parts = ln.split()
                # prefer full pose labels: cls cx cy w h + 3*K
                if len(parts) >= 8 and (len(parts) - 5) % 3 == 0:
                    Kcand = (len(parts) - 5) // 3
                    kp_counter[Kcand] += 1
                # stop early per file after a couple of lines
                if j >= 2:
                    break
    except Exception:
        continue

if not kp_counter:
    # Fallback: detect K from kpt-only (cls x1 y1 x2 y2 ...)
    kpt_only_counter = Counter()
    for i, fp in enumerate(label_txts[:2000]):
        try:
            with open(fp, 'r') as f:
                for j, ln in enumerate(f):
                    ln = ln.strip()
                    if not ln:
                        continue
                    parts = ln.split()
                    if len(parts) >= 3 and (len(parts) - 1) % 2 == 0:
                        Kcand = (len(parts) - 1) // 2
                        kpt_only_counter[Kcand] += 1
                    if j >= 2:
                        break
        except Exception:
            continue
    assert kpt_only_counter, 'Could not infer keypoints K from labels.'
    K = max(kpt_only_counter, key=kpt_only_counter.get)
else:
    K = max(kp_counter, key=kp_counter.get)

print(f"Inferred keypoints K (majority) = {K}")

# Heuristics to resolve images dirs for each split
def choose_images_dir(split: str):
    candidates = [
        root / 'images' / split,       # type-first
        root / split / 'images',       # split-first
    ]
    # derive from labels/<split>
    split_label_dirs = []
    for p in label_txts:
        p_norm = p.replace('\\', '/').lower()
        if f"/labels/{split}/" in p_norm:
            split_label_dirs.append(Path(p).parent)
    if split_label_dirs:
        ldir = split_label_dirs[0]
        candidates += [
            ldir.parent / 'images',                  # .../train/images
            ldir.parent.parent / 'images' / ldir.name # .../images/train
        ]
    for c in candidates:
        if c.exists():
            return str(c)
    return None

train_path = choose_images_dir('train')
val_path = choose_images_dir('val')
test_path = choose_images_dir('test')
print('Images dirs (raw):', {'train': train_path, 'val': val_path, 'test': test_path})

# Normalize to be RELATIVE to dataset root if possible, else absolute
root_abs = root.resolve()

def to_rel_or_abs(p):
    if not p:
        return None
    p_abs = Path(p)
    try:
        p_abs = p_abs.resolve()
    except Exception:
        p_abs = (root / p).resolve()
    try:
        return str(p_abs.relative_to(root_abs))
    except ValueError:
        return str(p_abs)

train_rel = to_rel_or_abs(train_path)
val_rel = to_rel_or_abs(val_path)
test_rel = to_rel_or_abs(test_path)
print('Images dirs (normalized):', {'train': train_rel, 'val': val_rel, 'test': test_rel})

assert train_rel and val_rel, (
    "Could not locate images/train and/or images/val. Ensure YOLO structure like:\n"
    "- dataset/images/train & dataset/labels/train\n"
    "- dataset/train/images & dataset/train/labels\n"
)

# Build YAML for pose
names = ['plate']
skeleton = [[0,1],[1,2],[2,3],[3,0]] if K==4 else []
flip_idx = list(range(K))

data = {
    'path': str(root_abs),
    'train': train_rel,
    'val': val_rel,
    'test': test_rel,
    'names': names,
    'kpt_shape': [K, 3],
    'skeleton': skeleton,
    'flip_idx': flip_idx,
}

yaml_path = root/'data.yaml'
with open(yaml_path, 'w') as f:
    yaml.safe_dump(data, f, sort_keys=False)

print('Wrote', yaml_path)
print(yaml.safe_dump(data, sort_keys=False))

# Clear label caches so Ultralytics rebuilds with the corrected kpt_shape
for cache_name in ['train.cache', 'val.cache']:
    cp = root / 'labels' / cache_name
    if cp.exists():
        try:
            os.remove(cp)
            print('Removed cache:', cp)
        except Exception as e:
            print('Failed to remove cache', cp, e)


In [ ]:
# 4) Validate & quick inference
from glob import glob
import os, yaml
import cv2, numpy as np
from ultralytics import YOLO

# locate last run more robustly
pose_runs = sorted(glob('runs/pose_plate/*')) if os.path.exists('runs/pose_plate') else []
run_dir = pose_runs[-1] if pose_runs else None
print('Last run:', run_dir)

# Load model from best/last if available, otherwise prefer local yolov8 pose weights
ckpt = None
if run_dir:
    for w in ['weights/best.pt', 'weights/last.pt']:
        p = os.path.join(run_dir, w)
        if os.path.exists(p):
            ckpt = p
            break

fallback = 'yolov8n-pose.pt'
print('Loading weights:', ckpt or fallback)
model = YOLO(ckpt or fallback)
print('Model task:', getattr(model, 'task', None))

# read data.yaml to find val images path
with open('./dataset/data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
val_path = data_cfg.get('val')
base_path = data_cfg.get('path', '.')
val_dir = os.path.join(base_path, val_path) if val_path else './dataset/val/images'

# pick a few images
val_imgs = []
for pat in ['*.jpg', '*.jpeg', '*.png', '*.*']:
    val_imgs = glob(os.path.join(val_dir, pat))
    if val_imgs:
        break
val_imgs = val_imgs[:6]
print('Previewing', len(val_imgs), 'images from', val_dir)


def _to_numpy(x):
    try:
        import torch
        if isinstance(x, np.ndarray):
            return x
        if isinstance(x, torch.Tensor):
            return x.detach().cpu().numpy()
    except Exception:
        pass
    return np.asarray(x)


def _order_points_four(pts: np.ndarray) -> np.ndarray:
    pts = np.asarray(pts, dtype='float32')
    if pts.shape[0] != 4:
        return pts
    s = pts.sum(axis=1)
    diff = pts[:, 1] - pts[:, 0]
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype='float32')


for imgp in val_imgs:
    im = cv2.imread(imgp)
    res = model.predict(source=im, imgsz=640, conf=0.5, iou=0.6, classes=[0], verbose=False, max_det=50)[0]

    # Draw boxes
    if res.boxes is not None and len(res.boxes) > 0:
        for b in res.boxes:
            x1,y1,x2,y2 = b.xyxy[0].int().cpu().tolist()
            cv2.rectangle(im, (x1,y1), (x2,y2), (0,255,0), 2)

    # Draw keypoints with CPU conversion and polygon
    if getattr(res, 'keypoints', None) is not None and res.keypoints.xy is not None:
        kxy = _to_numpy(res.keypoints.xy)
        kcf = _to_numpy(getattr(res.keypoints, 'conf', None)) if getattr(res.keypoints, 'conf', None) is not None else None
        for i in range(kxy.shape[0]):
            pts = kxy[i]
            if pts is None or pts.shape[0] < 4:
                continue
            mask = np.ones((pts.shape[0],), dtype=bool)
            if kcf is not None:
                mask = kcf[i] >= 0.1
            pts_vis = pts[mask]
            if pts_vis.shape[0] >= 4:
                pts4 = pts_vis[:4]
                ordered = _order_points_four(pts4)
                poly = ordered.reshape((-1,1,2)).astype(int)
                cv2.polylines(im, [poly], isClosed=True, color=(0,0,255), thickness=2)
                for px, py in ordered:
                    cv2.circle(im, (int(px), int(py)), 4, (0,255,255), -1)
            else:
                for px, py in pts[:4]:
                    cv2.circle(im, (int(px), int(py)), 4, (0,255,255), -1)

    cv2.imshow('preview', im)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()


In [ ]:
# 3) Train
from ultralytics import YOLO
from pathlib import Path
import torch
import os, glob

# Disable cloud sync/telemetry to reduce surprise downloads
try:
    from ultralytics.utils import SETTINGS
    SETTINGS.update({'sync': False})
except Exception:
    pass

yaml_path = Path('./dataset/data.yaml')
assert yaml_path.exists(), 'data.yaml not found; run the dataset YAML cell.'

# Remove unrelated YOLOv11/YOLO11 detection weights to avoid accidental selection
for pat in ['yolo11*.pt', 'yolov11*.pt']:
    for p in glob.glob(pat):
        try:
            os.remove(p)
            print('Removed unrelated weight:', p)
        except Exception as e:
            print('Could not remove', p, e)

model_name = 'yolov8n-pose.pt'  # nano pose model
model = YOLO(model_name)
print('Using base weights:', model_name, '| task:', getattr(model, 'task', None))

# Training options (up to 200 epochs with early stopping)
train_args = dict(
    data=str(yaml_path),
    imgsz=768,        # bigger image size helps pose accuracy
    epochs=200,       # train up to 200 epochs
    patience=30,      # early stop if no val improvement for 30 epochs
    batch=4,          # adjust if you hit OOM
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=2,        # keep low for laptops
    project='runs',
    name='pose_plate',
    exist_ok=True,
    pretrained=False,  # avoid triggering any default weight downloads
    cache=False,
    plots=True,       # save results.png and curves
    save_period=20,   # checkpoint every 20 epochs
    seed=42,
 )

results = model.train(**train_args)
print('Training done. Best:', results)

In [ ]:
# 3.b) Fine-tune from previous best.pt (creates a new run)
from ultralytics import YOLO
from pathlib import Path
import os, glob, torch

# Data YAML must exist
yaml_path = Path('./dataset/data.yaml')
assert yaml_path.exists(), 'data.yaml not found; ensure it exists before fine-tune.'

# Prefer manual checkpoint via env or local path if provided
manual_ckpt = os.environ.get('PLATE_WEIGHTS')  # e.g. ./weights/plate_pose_best.pt
ckpt = manual_ckpt if manual_ckpt and os.path.exists(manual_ckpt) else None

# Try default best path from main training run
if not ckpt:
    default_best = 'runs/pose_plate/weights/best.pt'
    if os.path.exists(default_best):
        ckpt = default_best

# Otherwise, search all best.pt under runs/**/weights and pick the newest
if not ckpt:
    bests = glob.glob('runs/**/weights/best.pt', recursive=True)
    if bests:
        try:
            bests.sort(key=lambda p: os.path.getmtime(p))
        except Exception:
            bests.sort()
        ckpt = bests[-1]
        print('Picked latest best.pt:', ckpt)

# Fallback to base pose weights
if not ckpt:
    ckpt = 'yolov8n-pose.pt'

print('Fine-tuning from:', ckpt)
model = YOLO(ckpt)

# Safe speed knobs
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    try:
        torch.set_float32_matmul_precision('high')
    except Exception:
        pass

# RAM usage controls
USE_RAM_CACHE = os.environ.get('YOLO_RAM_CACHE', '0') in ('1', 'true', 'True')
CACHE_MODE = 'ram' if USE_RAM_CACHE else False
WORKERS = int(os.environ.get('YOLO_WORKERS', '4'))  # lower a bit to reduce prefetch memory
print(f'Cache mode: {CACHE_MODE} | workers: {WORKERS}')

# Moderate, stable fine-tune config
train_args = dict(
    data=str(yaml_path),
    imgsz=640,           # increase detail for 4 keypoints
    epochs=40,          # short FT round
    patience=10,
    lr0=0.002, lrf=0.05,  # smaller LR for fine-tune
    batch=6,            # adjust to your VRAM
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=WORKERS,    # slightly lower to reduce RAM footprint
    project='runs',
    name='pose_plate_ft',  # new run name to keep old run intact
    exist_ok=True,
    pretrained=False,
    cache=CACHE_MODE,   # avoid huge RAM spikes by default; opt-in via YOLO_RAM_CACHE=1
    plots=False,        # reduce overhead
    save_period=0,      # save only best/last
    seed=42,
)

results = model.train(**train_args)
print('Fine-tune done. Best:', results)
